In [1]:
import os

In [2]:
%pwd

'e:\\Neoron\\Programming_Practice\\Machine_Learning_Project\\winequality_prediction\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Neoron\\Programming_Practice\\Machine_Learning_Project\\winequality_prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: str
    target_column: str
    mlflow_uri: str

In [6]:
'''os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/nasir.uddin.6314/WineQualityPrediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="nasir.uddin.6314"
os.environ["MLFLOW_TRACKING_PASSWORD"]="026054fecb7c525dea65edc373c6935c9a1332fc "'''

'os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/nasir.uddin.6314/WineQualityPrediction.mlflow"\nos.environ["MLFLOW_TRACKING_USERNAME"]="nasir.uddin.6314"\nos.environ["MLFLOW_TRACKING_PASSWORD"]="026054fecb7c525dea65edc373c6935c9a1332fc "'

In [7]:
from src.WineQualityPrediction.constants import *
from src.WineQualityPrediction.utils.common import read_yaml, create_directories
from src.WineQualityPrediction.utils.my_logging import logger
from src.WineQualityPrediction.utils.my_exception import CustomException

In [8]:
class ConfigurationManager:
    """
    Manages the configuration for the project by reading YAML files
    and providing structured configurations.

    Attributes:
        config (ConfigBox): Parsed configuration file.
        params (ConfigBox): Parsed parameters file.
        schema (ConfigBox): Parsed schema file.
    """

    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH,
                 schema_filepath: Path = SCHEMA_FILE_PATH):
        
        """
        Initializes the ConfigurationManager by loading YAML files and creating directories.

        Args:
            config_filepath (Path): Path to the main configuration file.
            params_filepath (Path): Path to the parameters configuration file.
            schema_filepath (Path): Path to the schema configuration file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/nasir.uddin.6314/WineQualityPrediction.mlflow"


        )
        return model_evaluation_config    


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from src.WineQualityPrediction.utils.common import *
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from typing import Dict

In [10]:
class ModelEvaluation:
    """
    Initializes the ModelEvaluation class with a given configuration.

    Parameters:
        config (ModelEvaluationConfig): An object containing configuration parameters 
                                        for model evaluation, including paths for data, 
                                        model, MLflow URI, and metric file storage.
    """    
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    
    
    def eval_metrics(self,actual, pred):
        """
        Calculates evaluation metrics for the model predictions.

        Parameters:
            actual (array-like): Ground truth target values.
            pred (array-like): Predicted target values from the model.

        Returns:
            tuple: A tuple containing:
                - `rmse` (float): Root Mean Squared Error.
                - `mae` (float): Mean Absolute Error.
                - `r2` (float): R-squared score.

        Dependencies:
            - numpy: Used for mathematical calculations (e.g., square root).
            - sklearn.metrics: For calculating regression metrics.
        """        
        
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    
    
    
    def log_into_mlflow(self):
        """
        Logs evaluation metrics and model details into MLflow and optionally registers the model.
        This method sets up MLflow tracking and authenticates using environment variables.
        """
        # Set MLflow tracking environment variables
        os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/nasir.uddin.6314/WineQualityPrediction.mlflow"
        os.environ["MLFLOW_TRACKING_USERNAME"] = "nasir.uddin.6314"
        os.environ["MLFLOW_TRACKING_PASSWORD"] = "026054fecb7c525dea65edc373c6935c9a1332fc"

        # Configure MLflow registry URI
        mlflow.set_registry_uri(os.getenv("MLFLOW_TRACKING_URI"))
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Split test data into features (X) and target (Y)
        test_x = test_data.drop(columns=[self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        # Start an MLflow run
        with mlflow.start_run(run_name="ModelEvaluationRun"):
            # Predict target values using the loaded model
            predicted_qualities = model.predict(test_x)

            # Evaluate metrics
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            # Save evaluation metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            # save this score as a json file 
            save_json(self.config.metric_file_name, scores)

            # Log parameters and metrics into MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Log and optionally register the model
            if tracking_url_type_store != "file":
                # Register the model if the tracking store supports it
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                # Log the model without registering
                mlflow.sklearn.log_model(model, "model")

            print("Model evaluation metrics and model details logged successfully.")


In [11]:
try: 
    config_manager = ConfigurationManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except CustomException as e:
    raise e

[2025-01-15 19:30:18] - INFO - Yaml file read successfully
[2025-01-15 19:30:18] - INFO - Yaml file read successfully
[2025-01-15 19:30:18] - INFO - Yaml file read successfully
[2025-01-15 19:30:18] - INFO - Directory created at artifacts
[2025-01-15 19:30:18] - INFO - Directory created at artifacts/model_evaluation
[2025-01-15 19:30:19] - INFO - JSON data saved to artifacts/model_evaluation/metrics.json
2025/01/15 19:30:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/01/15 19:30:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 3
Created version '3' of model 'ElasticnetModel'.


Model evaluation metrics and model details logged successfully.
🏃 View run ModelEvaluationRun at: https://dagshub.com/nasir.uddin.6314/WineQualityPrediction.mlflow/#/experiments/0/runs/c9e481ae5d7a4461908a97bea0a2d809
🧪 View experiment at: https://dagshub.com/nasir.uddin.6314/WineQualityPrediction.mlflow/#/experiments/0
